### 국회의원 정보 스크래핑
* 이름, ID 추출
* ID를 사용해서 상세페이지로 요청 보내기
* DataFrame에 저장하고, DB에 저장
* 시각화

In [15]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do'
req_param = {
    #?currentPage=1&rowPerPage=500
    "currentPage":1,
    "rowPerPage":500
}

res = requests.get(url, params=req_param)
print(res.status_code)

if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    atag_list = soup.select("div.memberna_list dl dt a")
    
    member_list = []    
    for atag in atag_list:

        member_dict = {}
        
        member_dict['name'] = atag.text  #<a href="">둘리</a>
        
        href = atag['href']
        matched = re.search(r'(\d+)', href)
        if matched:
            member_dict['id'] = matched.group(0)
        
        #print(member_dict)
        member_list.append(member_dict)

print(len(member_list))
member_list[0:2]

200
297


[{'name': '강기윤', 'id': '9771230'}, {'name': '강대식', 'id': '9771142'}]

In [41]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

print('----- 시작')

#모든 국회의원 정보를 저장할 list 선언
member_list = []

for idx,member in enumerate(member_list[0:2],1):
    print(f'---> 번호 {idx} <------')
    member_id = member['id']
    detail_url = f'https://www.assembly.go.kr/assm/memPop/memPopup.do?dept_cd={member_id}'
    
    res = requests.get(detail_url)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        #1명의 국회의원 정보를 저장할 dict를 선언
        member_dict = dict()
        
        dt_list = []
        for dt in soup.select("div.info_mna dl.pro_detail dt"):
            dt_list.append(dt.text)
        
        dd_list = []
        for dd in soup.select("div.info_mna dl.pro_detail dd"):
            dd_text = dd.text.strip()
            
            #Pattern 객체생성
            reg_exr = re.compile(r'[\n\r\t]')
            #sub()는 Pattern과 일치하는 문자을 empty string으로 대체해라
            #replace()는 white space를 empty string으로 변경해라
            clean_text = reg_exr.sub('', dd_text).replace(" ","")
            dd_list.append(clean_text)
        
        member_dict = dict(zip(dt_list,dd_list))
        #print(member_dict)
        
        for div_tag in soup.select("div.info_mna div.profile"):
            #print(div_tag)
            member_dict["이름"] = div_tag.find('h4').text
            
            img_src = div_tag.select('img')[0]['src']
            img_url = urljoin(detail_url, img_src)
            member_dict["이미지"] = img_url
            
            birth_date = div_tag.select_one("li:nth-of-type(4)").text
            member_dict["생년월일"] = birth_date
            
        member_list.append(member_dict)   

print(len(member_list))
member_list[0:2]
print('----- 끝')        

----- 시작
---> 번호 1 <------
{'정당': '국민의힘', '선거구': '경남창원시성산구', '소속위원회': '보건복지위원회', '당선횟수': '재선(19대,21대)', '사무실 전화': '02-784-1751', '사무실 호실': '의원회관937호', '홈페이지': 'http://blog.naver.com/ggotop', '이메일': 'ggotop@naver.com', '보좌관': '김홍광,한영애', '비서관': '지상훈,최광림', '비서': '김영록,안효상,이유진,홍지형,김지훈', '취미, 특기': '', '의원실 안내': '', '이름': '강기윤', '이미지': 'https://www.assembly.go.kr/photo/9771230.jpg', '생년월일': '1960-06-04'}
---> 번호 2 <------
{'정당': '국민의힘', '선거구': '대구동구을', '소속위원회': '국방위원회', '당선횟수': '초선(21대)', '사무실 전화': '02-784-5275', '사무실 호실': '의원회관341호', '홈페이지': '', '이메일': 'kds21341@naver.com', '보좌관': '박홍규,정운태', '비서관': '유진영,윤미라', '비서': '박순권,김광연,윤선영,김현정,송민욱', '취미, 특기': '', '의원실 안내': '', '이름': '강대식', '이미지': 'https://www.assembly.go.kr/photo/9771142.jpg', '생년월일': '1959-11-02'}
